<a href="https://colab.research.google.com/github/shreekanthsenthil/Malicious-URLs-Detection-using-Machine-Learning/blob/master/Detect_Malicious_URLs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import csv
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/content/data.csv')
df.head()

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad


In [3]:
print('Before Removing Duplicates')
print('Total : ' , len(df))
print('Good : ' , len(df[df['label'] == 'good']))
print('Bad : ' , len(df[df['label'] == 'bad']))
df.drop_duplicates(subset='url',keep=False,inplace=True)
print('After Removing Duplicates')
df.reset_index(drop=True,inplace=True)
print('Total : ' , len(df))
print('Good : ' , len(df[df['label'] == 'good']))
print('Bad : ' , len(df[df['label'] == 'bad']))

Before Removing Duplicates
Total :  420464
Good :  344821
Bad :  75643
After Removing Duplicates
Total :  403155
Good :  344778
Bad :  58377


In [4]:
def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

df = encode_categorical(df, ['label'])


The  label is encoded 


In [5]:
df1 = np.array(df)
random.shuffle(df1)
print(df1)

[['diaryofagameaddict.com' 0]
 ['espdesign.com.au' 0]
 ['iamagameaddict.com' 0]
 ...
 ['truthaboutscientology.com/stats/by-name/d/david-van-den-berg.html' 1]
 ['website-tools.net/google-keyword/word/catherine+faylen' 1]
 ['vmeste-hudeem.ru/yr387n3' 0]]


In [0]:
y = [d[1] for d in df1]
url = [d[0] for d in df1]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(url)

In [7]:
print(X[0])

  (0, 88809)	0.11771921948862656
  (0, 98787)	0.9930469200208963


In [0]:
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.2,shuffle=True,stratify=y)

In [9]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=0).fit(x_train, y_train)
accuracy = LR.score(x_test, y_test)
print('Accuracy : ' ,accuracy)

Accuracy :  0.9827857771824733


In [10]:
predict = LR.predict(x_test)
conf_mat = confusion_matrix(y_test, predict)
print(conf_mat)
print(classification_report(y_test, predict)) 

[[21369  1147]
 [  241 57874]]
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     22516
           1       0.98      1.00      0.99     58115

    accuracy                           0.98     80631
   macro avg       0.98      0.97      0.98     80631
weighted avg       0.98      0.98      0.98     80631



.  | Bad Predicted | Good Predicted
--- | --- | ---
Bad Actual | 21369 | 1147
Good Actual | 241 | 57874

In [0]:
pickle.dump(LR, open("LR_saved_model.pickle", "wb"))
pickle.dump(vectorizer, open("tfidf.pickle", "wb"))